### Cleaning up data and and creating master data frame / CSV file

In [124]:
import numpy as np
import pandas as pd
from pathlib import Path
%matplotlib inline

In [125]:
# It is not working for Carlos, so need to run it twice
import numpy as np
import pandas as pd
from pathlib import Path
%matplotlib inline
import hvplot.pandas

## All the data bases: 
* ### Static:
 * rent_stats_df: Rent statistics by City and State (% Rent Change Since Feb, Avg Monthly Rent Last 12M, Last 12M Annualized Rent, Monthly Rent Last, Rent CAGR since 2014-01, Rent Standard Deviation)
 * lon_lat_density_df: lat/lon and populaiton information on each city / state City (State, Population, Density, Zip Code, lat,lon)
 * sale_stats_df:  Sale prices statistics by City and State (City, State, % Sale Price Change Since Feb, Avg Sale Price Last 12M, Sale Price Last, Sale Pr CAGR GFC, Sale Pr Standard Deviation)
 * median_income_df: Income info for each City / State (Median income (dollars)	Mean income (dollars))
 * census_stats_df: Population data for different MSAs (City, State, Population 2019, Population 2010, Population 10 year CAGR)
 * combined_df: Combines all of the above; performs an outer join in case we want to use different data for different purposes

* ### Historic:
 * rental_price_df: Historical information for rents since 2014
 * house_price_df: Historical information for sale prices since 2014
 * census_df: Annual population Estimates (2010-2020) for multiple MSAs



## Sale Price Data Base
### Cleaning and Re arranging

In [126]:
# read the house_prices_msa.csv. this file will be used for sale price data
file_name='house_prices_msa.csv'
csv_file=Path(f"../Data/{file_name}")
house_price_df= pd.read_csv(csv_file)
# house_price_df.head(2)

# Cleaning and re arranging data 
house_price_df.set_index(house_price_df['RegionName'],inplace=True)
house_price_df=house_price_df.drop(columns=['RegionID','SizeRank', 'RegionName', 'RegionType', 'StateName'])


# Create columns with city and state to allow more flexibility when showing data
house_price_df.index.rename('MSA_interim', inplace=True)
house_price_df.reset_index(inplace=True)

# DELETE house_price_df.rename(columns={'index':'MSA'}, inplace= True)
house_price_df[['city_interim','State']] = house_price_df['MSA_interim'].str.split(", ", n = 1, expand=True) # Split City from text string

# Keep first city in list; e.g Dallas-Fort Worth will be dallas
house_price_df[['City','Extra1']] = house_price_df['city_interim'].str.split("-", n = 1, expand=True) # Split Cities from text string

# Create unique identifier
house_price_df['MSA']=house_price_df['City']+', '+house_price_df['State']

# Replace United States that is NAN. Confirmed is the only null with house_price_df['MSA'].isna().sum()
house_price_df['MSA'].fillna('United States',inplace=True)

# Change name of MSa to Date and create an index
# DELETE house_price_df.rename(columns={'MSA':'Date'},inplace=True)
# DELETE: house_price_df.set_index('Date',inplace=True)

house_price_df=house_price_df.drop(columns=['city_interim','State', 'City', 'Extra1','MSA_interim'])
house_price_df.set_index('MSA',inplace=True)

# Transpose dataframe
house_price_df=house_price_df.transpose()
# Drop the name that would look as the name of the index otherwise
house_price_df.columns.name = None
# set index
house_price_df.index.rename('Date',inplace=True)


house_price_df.head(5)
# region_id_df.plot(legend=False) 


,United States,"New York, NY","Los Angeles, CA","Chicago, IL","Dallas, TX","Philadelphia, PA","Houston, TX","Washington, DC","Miami, FL","Atlanta, GA",...,"Winnemucca, NV","Sweetwater, TX","Andrews, TX","Zapata, TX","Vermillion, SD","Lamesa, TX","Craig, CO","Pecos, TX","Vernon, TX","Ketchikan, AK"
Date,,,,,,,,,,,,,,,,,,,,,
1/31/1996,107630.0,187842.0,183929.0,164647.0,114406.0,121233.0,110773.0,178034.0,109403.0,120291.0,...,NaN,NaN,NaN,NaN,NaN,NaN,60501.0,NaN,NaN,NaN
2/29/1996,107657.0,187403.0,184185.0,164345.0,114471.0,121078.0,110849.0,177811.0,109533.0,120519.0,...,NaN,NaN,NaN,NaN,NaN,NaN,60725.0,NaN,NaN,NaN
3/31/1996,107707.0,187125.0,184205.0,163946.0,114634.0,120937.0,110858.0,177681.0,109670.0,120740.0,...,NaN,NaN,NaN,NaN,NaN,NaN,60961.0,NaN,NaN,NaN
4/30/1996,107834.0,186592.0,184312.0,163493.0,114962.0,120693.0,111007.0,177407.0,109917.0,121229.0,...,NaN,NaN,NaN,NaN,NaN,NaN,61420.0,NaN,NaN,NaN
5/31/1996,107977.0,186274.0,184286.0,162886.0,115314.0,120527.0,111148.0,177288.0,110132.0,121711.0,...,NaN,NaN,NaN,NaN,NaN,NaN,61747.0,NaN,NaN,NaN


In [127]:
# Calcuate variables from house database 

# Change in price since February, prior to COVID hit in March
end_date='9/30/2020'
start_date='2/29/2020'
change_since_feb=house_price_df.loc[end_date]/house_price_df.loc[start_date]-1


# sale_stats_df=pd.DataFrame([[change_since_feb,avg_last_12m]],columns=['% Change Since Feb','AVG Last 12M'])
sale_stats_df=pd.DataFrame(change_since_feb,columns=['% Sale Price Change Since Feb'])

# Similar change but for prior year (no covid); to be used as control
# Change in price since February, prior to COVID hit in March
end_date='9/30/2019'
start_date='2/28/2019'
change_since_feb_2019=house_price_df.loc[end_date]/house_price_df.loc[start_date]-1
# add column
sale_stats_df['% Sale Price Change Since Feb 2019']=change_since_feb_2019





# df.index = df["Index Title"]
# Last 12 Months average: to show cost and avoid intra month volatility
avg_last_12m=house_price_df.iloc[-12:,:].mean()
sale_stats_df['Avg Sale Price Last 12M']=avg_last_12m

# Most Recent Data Point
last_price=house_price_df.iloc[-1,:]
sale_stats_df['Sale Price Last']=last_price


# Calculate CAGR: Min post GFC to Most Recent Date
# Add a column with most recent date
# Create a column that i will drop with last_date - used to calculate number of periods
sale_stats_df['last_date']=house_price_df.index[-1]
sale_stats_df['last_date']=pd.to_datetime(sale_stats_df['last_date'], errors='coerce')


#  Found the minimum data point after the GFC for each MSA
# Create a column that i will drop with last_date - used to calculate number of periods
sale_stats_df['gfc_min']=house_price_df.loc['1/31/2008':].idxmin()
sale_stats_df['gfc_min']=pd.to_datetime(sale_stats_df['gfc_min'], errors='coerce')

# Populate the minimum value from series
sale_stats_df['gfc_min_value']=house_price_df.loc['1/31/2008':].min()


# calculate the number of periods
sale_stats_df['months_diff'] = sale_stats_df['last_date'].dt.to_period('M').astype(int)-sale_stats_df['gfc_min'].dt.to_period('M').astype(int)

# Actual CAGR
period=12
sale_stats_df['Sale Pr CAGR GFC']=(sale_stats_df['Sale Price Last']/sale_stats_df['gfc_min_value'])**(period*1/sale_stats_df['months_diff'])-1


# Calculate Standard Deviation and I annualize
monthly_returns_sales_df=house_price_df.pct_change()
monthly_std_sales_df=monthly_returns_sales_df.std()#* np.sqrt(12)
sale_stats_df['Sale Pr Standard Deviation']=monthly_returns_sales_df.std()* np.sqrt(12)

# Create columns with city and state to allow more flexibility when showing data
# sale_stats_df[['City','State']] = sale_stats_df['MSA'].str.split(", ", n = 1, expand=True) # Split City from text string
sale_stats_df.reset_index(inplace=True)

# Delete the existing name of the index
# sale_stats_df.columns.name = None
# Replace name of column that was mislabeled after transposing / Rename index
sale_stats_df.rename(columns={'index':'MSA'}, inplace= True)


sale_stats_df[['City','State']] = sale_stats_df['MSA'].str.split(", ", n = 1, expand=True) # Split City from text string
sale_stats_df.set_index('MSA',inplace=True)


sale_stats_df


,% Sale Price Change Since Feb,% Sale Price Change Since Feb 2019,Avg Sale Price Last 12M,Sale Price Last,last_date,gfc_min,gfc_min_value,months_diff,Sale Pr CAGR GFC,Sale Pr Standard Deviation,City,State
MSA,,,,,,,,,,,,
United States,0.041127,0.021589,251932.416667,259906.0,2020-09-30,2012-02-29,162675.0,103,0.056108,0.015617,United States,None
"New York, NY",0.031853,0.006435,485123.166667,497090.0,2020-09-30,2012-09-30,372880.0,96,0.036593,0.018566,New York,NY
"Los Angeles, CA",0.045627,0.011159,686194.416667,711361.0,2020-09-30,2012-03-31,389805.0,102,0.073333,0.031222,Los Angeles,CA
"Chicago, IL",0.030465,0.009426,247694.583333,253512.0,2020-09-30,2012-09-30,178666.0,96,0.044707,0.022278,Chicago,IL
"Dallas, TX",0.036259,0.013690,263875.916667,270907.0,2020-09-30,2012-01-31,146517.0,104,0.073495,0.012663,Dallas,TX
...,...,...,...,...,...,...,...,...,...,...,...,...
"Lamesa, TX",0.020283,0.039865,76011.333333,77216.0,2020-09-30,2011-01-31,43095.0,116,0.062188,0.013307,Lamesa,TX
"Craig, CO",0.002653,0.004033,187330.500000,187430.0,2020-09-30,2011-06-30,134260.0,111,0.036726,0.014049,Craig,CO
"Pecos, TX",0.027741,0.070362,127703.000000,130703.0,2020-09-30,2015-12-31,78791.0,57,0.112437,0.012827,Pecos,TX


In [128]:
# re arrange order and drop columns we do not need
sale_stats_df=sale_stats_df[['City','State','% Sale Price Change Since Feb', '% Sale Price Change Since Feb 2019','Avg Sale Price Last 12M', 'Sale Price Last', 'Sale Pr CAGR GFC','Sale Pr Standard Deviation']]
sale_stats_df.head(5)

# sale_stats_df[sale_stats_df['City']=='Longview']

,City,State,% Sale Price Change Since Feb,% Sale Price Change Since Feb 2019,Avg Sale Price Last 12M,Sale Price Last,Sale Pr CAGR GFC,Sale Pr Standard Deviation
MSA,,,,,,,,
United States,United States,None,0.041127,0.021589,251932.416667,259906.0,0.056108,0.015617
"New York, NY",New York,NY,0.031853,0.006435,485123.166667,497090.0,0.036593,0.018566
"Los Angeles, CA",Los Angeles,CA,0.045627,0.011159,686194.416667,711361.0,0.073333,0.031222
"Chicago, IL",Chicago,IL,0.030465,0.009426,247694.583333,253512.0,0.044707,0.022278
"Dallas, TX",Dallas,TX,0.036259,0.013690,263875.916667,270907.0,0.073495,0.012663


## Latitude, Longitude, Population Data Base
### Cleaning and Re arranging

In [129]:
file_name='uscities.csv'
csv_file=Path(f"../Data/{file_name}")
lon_lat_density_df= pd.read_csv(csv_file)
lon_lat_density_df.head(2)

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220.0,10715.0,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12750807.0,3276.0,polygon,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491


In [130]:
# read the uscities csv. this file will be used for the longitude and latitude, as well as the density stats and population filters
file_name='uscities.csv'
csv_file=Path(f"../Data/{file_name}")
lon_lat_density_df= pd.read_csv(csv_file)
# lon_lat_density_df.head(2)

# getting first zipcode from list of zipcodes and create primary MSA from stateid and city
lon_lat_density_df['Zip Code'] = lon_lat_density_df['zips'].str.split(", ", n = 1, expand=True)
lon_lat_density_df['Zip Code'] = lon_lat_density_df['zips'].str[:5]

# Creating Unique ID: city + State
lon_lat_density_df['MSA']= lon_lat_density_df['city_ascii'].str[:]+', '+lon_lat_density_df['state_id'].str[:2]

# Rename Columns
lon_lat_density_df.rename(columns={'city_ascii':'City','state_id':'State','lng':'lon','population':'Population','density':'Density'},inplace=True)

# dropping unnecessary columns, reording columns and setting index
lon_lat_density_df=lon_lat_density_df[['MSA','City','State', 'Population','Density','Zip Code','lat', 'lon']]
lon_lat_density_df.drop_duplicates()
lon_lat_density_df=lon_lat_density_df.set_index('MSA')
lon_lat_density_df.head(5)

# This data set has informatio for more cities than sales and rental databases
# (longlatdensity_df[longlatdensity_df['population']>100000])

,City,State,Population,Density,Zip Code,lat,lon
MSA,,,,,,,
"New York, NY",New York,NY,18713220.0,10715.0,11229,40.6943,-73.9249
"Los Angeles, CA",Los Angeles,CA,12750807.0,3276.0,90291,34.1139,-118.4068
"Chicago, IL",Chicago,IL,8604203.0,4574.0,60018,41.8373,-87.6862
"Miami, FL",Miami,FL,6445545.0,5019.0,33129,25.7839,-80.2102
"Dallas, TX",Dallas,TX,5743938.0,1526.0,75287,32.7936,-96.7662


## Rent Data Base
### Cleaning and Re arranging

In [131]:
# read the house_prices_msa.csv. this file will be used for sale price data
file_name='rental_prices_msa.csv'
csv_file=Path(f"../Data/{file_name}")
rental_price_df= pd.read_csv(csv_file)
rental_price_df.head(2)

# Cleaning and re arranging data 
rental_price_df.set_index(rental_price_df['RegionName'],inplace=True)
rental_price_df=rental_price_df.drop(columns=['RegionID','SizeRank','RegionName'])

rental_price_df.index.rename('MSA_interim', inplace=True)
rental_price_df.reset_index(inplace=True)



# Create State and City - will delete after i create unique ID
rental_price_df[['city_interim','State']] = rental_price_df['MSA_interim'].str.split(", ", n = 1, expand=True) # Split City from text string

# Keep first city in list; e.g Dallas-Fort Worth will be dallas
rental_price_df[['City','Extra1']] = rental_price_df['city_interim'].str.split("-", n = 1, expand=True) # Split Cities from text string

# Create nique identifier
rental_price_df['MSA']=rental_price_df['City']+', '+rental_price_df['State']

# Replace United States that is NAN. Confirmed is the only null with house_price_df['MSA'].isna().sum()
rental_price_df['MSA'].fillna('United States',inplace=True)

rental_price_df.set_index('MSA',inplace=True)

# Delete columns i do not need
rental_price_df=rental_price_df.drop(columns=['city_interim','State', 'City', 'Extra1','MSA_interim'])

# Change name of MSa to Date and create an index

# Transpose data frame
rental_price_df=rental_price_df.transpose()
# Delete old name from index
rental_price_df.columns.name = None
# Give a new name ot the index
rental_price_df.index.rename('Date',inplace=True)

# DELETE rental_price_df.reset_index(inplace=True)

# DELETE Drop duplicated columns
# DELETE rental_price_df=rental_price_df.loc[:,~rental_price_df.columns.duplicated()]

# DELETE rental_price_df.index.dt.to_period('M').astype(int)
rental_price_df.head(5)



,United States,"New York, NY","Los Angeles, CA","Chicago, IL","Dallas, TX","Philadelphia, PA","Houston, TX","Washington, DC","Miami, FL","Atlanta, GA",...,"Melbourne, FL","Chattanooga, TN","Spokane, WA","Provo, UT","Durham, NC","Port St. Lucie, FL","Fort Collins, CO","Boulder, CO","Greeley, CO","Gainesville, GA"
Date,,,,,,,,,,,,,,,,,,,,,
2014-01,1354.0,2296.0,1857.0,1527.0,1222.0,1364.0,1272.0,1893.0,1571.0,1139.0,...,986.0,914.0,857.0,991.0,1173.0,1157.0,1239.0,1522.0,1252.0,978.0
2014-02,1360.0,2307.0,1870.0,1532.0,1228.0,1367.0,1282.0,1897.0,1578.0,1144.0,...,991.0,NaN,858.0,997.0,1175.0,1165.0,1252.0,1534.0,1262.0,981.0
2014-03,1366.0,2318.0,1884.0,1537.0,1234.0,1369.0,1292.0,1901.0,1586.0,1148.0,...,997.0,NaN,859.0,1004.0,1177.0,1173.0,1265.0,1547.0,1272.0,984.0
2014-04,1372.0,2329.0,1897.0,1542.0,1240.0,1372.0,1302.0,1905.0,1594.0,1152.0,...,1003.0,938.0,860.0,1010.0,1178.0,1181.0,1278.0,1560.0,1282.0,987.0
2014-05,1378.0,2339.0,1910.0,1546.0,1245.0,1374.0,1311.0,1909.0,1602.0,1157.0,...,1008.0,945.0,861.0,1016.0,1180.0,1189.0,1291.0,1572.0,1292.0,991.0


In [132]:
# Calcuate variables from house database 

# Change in price since February, prior to COVID hit in March
end_date=rental_price_df.index[-1]
start_date='2020-02'
change_since_feb=rental_price_df.loc[end_date]/rental_price_df.loc[start_date]-1

# sale_stats_df=pd.DataFrame([[change_since_feb,avg_last_12m]],columns=['% Change Since Feb','AVG Last 12M'])
rent_stats_df=pd.DataFrame(change_since_feb,columns=['% Rent Change Since Feb'])

# Similar change but for prior year (no covid); to be used as control
end_date=rental_price_df.index[-1-12] #12 months perior to last month
start_date='2019-02'
change_since_feb_2019=rental_price_df.loc[end_date]/rental_price_df.loc[start_date]-1
# add column
rent_stats_df['% Rent Change Since Feb 2019']=change_since_feb_2019


# df.index = df["Index Title"]
# Last 12 Months average: to show cost and avoid intra month volatility
avg_last_12m=rental_price_df.iloc[-12:,:].mean()
rent_stats_df['Avg Monthly Rent Last 12M']=avg_last_12m

# Do the same for 2019
avg_2019= rental_price_df.loc['2019-01':'2019-12',:].mean()
rent_stats_df['Avg Monthly Rent 2019']=avg_2019

# Annualized Last 12 Months average: to calculate affordability
rent_stats_df['Last 12M Annualized Rent']=avg_last_12m*12
rent_stats_df['2019 Annualized Rent']=avg_2019*12


# Most Recent Data Point
last_price=rental_price_df.iloc[-1,:]
rent_stats_df['Monthly Rent Last']=last_price


# Calculate CAGR: 2014-01 or oldest data point if 2014-01 not available
# Add a column with most recent date
# Create a column that i will drop with last_date - used to calculate number of periods
rent_stats_df['last_date']=rental_price_df.index[-1]
rent_stats_df['last_date']=pd.to_datetime(rent_stats_df['last_date'], errors='coerce')

# Found the minimum data point after the GFC for each MSA
# Create a column that i will drop with last_date - used to calculate number of periods
rent_stats_df['oldest_date']=rental_price_df.index[0] # i need to populate the right data point
# rent_stats_df['oldest_date1']=rental_price_df.apply(pd.Series.first_valid_index)

rent_stats_df['oldest_date']=pd.to_datetime(rent_stats_df['oldest_date'], errors='coerce')
# s.loc[~s.isnull()].iloc[0] 


# Populate the minimum value from series
# make this date variable // ask
rent_stats_df['oldest_value']= rental_price_df.iloc[0] 
##rental_price_df.iloc[rental_price_df.apply(pd.Series.first_valid_index)]     #rental_price_df.loc[rental_price_df.notna().idxmax()] # rental_price_df.loc[rental_price_df.apply(pd.Series.first_valid_index)] 
#rental_price_df.loc[rental_price_df.apply(pd.Series.first_valid_index)]


# calculate the number of periods
rent_stats_df['months_diff'] = rent_stats_df['last_date'].dt.to_period('M').astype(int)-rent_stats_df['oldest_date'].dt.to_period('M').astype(int)

# Actual CAGR
period=12
rent_stats_df['Rent CAGR since 2014-01']=(rent_stats_df['Monthly Rent Last']/rent_stats_df['oldest_value'])**(period*1/rent_stats_df['months_diff'])-1


# Calculate Standard Deviation and I annualize
monthly_rent_change_df=rental_price_df.pct_change()
monthly_std_sales_df=monthly_rent_change_df.std()*np.sqrt(12)
rent_stats_df['Rent Standard Deviation']=monthly_returns_sales_df.std()


# Create columns with city and state to allow more flexibility when showing data
# sale_stats_df[['City','State']] = sale_stats_df['MSA'].str.split(", ", n = 1, expand=True) # Split City from text string
rent_stats_df.reset_index(inplace=True)
rent_stats_df.rename(columns={'index':'MSA'}, inplace= True)
rent_stats_df[['City','State']] = rent_stats_df['MSA'].str.split(", ", n = 1, expand=True) # Split City from text string
rent_stats_df.set_index('MSA',inplace=True)


# re arrange order and drop columns we do not need
rent_stats_df=rent_stats_df[['City', 'State', '% Rent Change Since Feb','% Rent Change Since Feb 2019','2019 Annualized Rent','Avg Monthly Rent 2019', 'Avg Monthly Rent Last 12M', 'Last 12M Annualized Rent','Monthly Rent Last' ,'Rent CAGR since 2014-01','Rent Standard Deviation']]

rent_stats_df.head(5)

,City,State,% Rent Change Since Feb,% Rent Change Since Feb 2019,2019 Annualized Rent,Avg Monthly Rent 2019,Avg Monthly Rent Last 12M,Last 12M Annualized Rent,Monthly Rent Last,Rent CAGR since 2014-01,Rent Standard Deviation
MSA,,,,,,,,,,,
United States,United States,None,0.005875,0.020556,20106.0,1675.500000,1703.916667,20447.0,1712.0,0.035816,0.004508
"New York, NY",New York,NY,-0.022710,0.015043,32189.0,2682.416667,2670.916667,32051.0,2625.0,0.020290,0.005360
"Los Angeles, CA",Los Angeles,CA,0.000000,0.018496,30186.0,2515.500000,2544.166667,30530.0,2545.0,0.048411,0.009013
"Chicago, IL",Chicago,IL,-0.003939,0.015464,21155.0,1762.916667,1775.083333,21301.0,1770.0,0.022398,0.006431
"Dallas, TX",Dallas,TX,0.005758,0.019029,18506.0,1542.166667,1564.750000,18777.0,1572.0,0.038502,0.003656


## Median Income Data Base
### Cleaning and Re arranging

In [133]:
file_name='household_income_msa_2019.csv'
csv_file=Path(f"../Data/{file_name}")
median_income_df= pd.read_csv(csv_file)
median_income_df.head(2)

,Label,"Aberdeen, WA Micro Area!!Households!!Estimate","Aberdeen, WA Micro Area!!Households!!Margin of Error","Aberdeen, WA Micro Area!!Families!!Estimate","Aberdeen, WA Micro Area!!Families!!Margin of Error","Aberdeen, WA Micro Area!!Married-couple families!!Estimate","Aberdeen, WA Micro Area!!Married-couple families!!Margin of Error","Aberdeen, WA Micro Area!!Nonfamily households!!Estimate","Aberdeen, WA Micro Area!!Nonfamily households!!Margin of Error","Abilene, TX Metro Area!!Households!!Estimate",...,"Yuma, AZ Metro Area!!Nonfamily households!!Estimate","Yuma, AZ Metro Area!!Nonfamily households!!Margin of Error","Zanesville, OH Micro Area!!Households!!Estimate","Zanesville, OH Micro Area!!Households!!Margin of Error","Zanesville, OH Micro Area!!Families!!Estimate","Zanesville, OH Micro Area!!Families!!Margin of Error","Zanesville, OH Micro Area!!Married-couple families!!Estimate","Zanesville, OH Micro Area!!Married-couple families!!Margin of Error","Zanesville, OH Micro Area!!Nonfamily households!!Estimate","Zanesville, OH Micro Area!!Nonfamily households!!Margin of Error"
0,Total,"29,920","±1,485","19,664","±1,605",N,N,"10,256","±1,630","61,750",...,"20,403","±2,093","33,342","±1,373","21,489","±1,465",N,N,"11,853","±1,455"
1,"Less than $10,000",5.60%,±2.0,2.50%,±1.9,N,N,12.00%,±5.4,7.10%,...,16.70%,±5.0,7.10%,±2.3,2.80%,±1.9,N,N,15.60%,±5.4


In [134]:
# Using median income CSV files from EH notebook. 
# data cleanup using EH notebook. Differences are: (created new medican_income_affdf for affordability df calc)

# read the house_prices_msa.csv. this file will be used for sale price data
file_name='household_income_msa_2019.csv'
csv_file=Path(f"../Data/{file_name}")
median_income_df= pd.read_csv(csv_file)
median_income_df.head(2)

# Transpose the column/headers
median_income_df = median_income_df.transpose() 

#grab the first row for the header
new_header = median_income_df.iloc[0] 

#take the data less the header row
median_income_df = median_income_df[1:] 

#set the header row as the df header
median_income_df.columns = new_header 

# Use only median and mean income data columns
median_income_df = median_income_df[['Median income (dollars)','Mean income (dollars)']] 
median_income_df.reset_index(inplace=True)

# Split MSA from text string
median_income_df[['MSA','extra']] = median_income_df['index'].str.split(",", n = 1, expand=True) 

# Split State from remaining text string
median_income_df[['State','extra2']] = median_income_df['extra'].str.split(n = 1, expand=True) 

# Split Primary MSA from first text string
median_income_df[['Primary MSA1','Secondary MSA']] = median_income_df['MSA'].str.split("-", n = 1, expand=True) 

# Split Primary MSA from first text string
median_income_df[['Primary State','Secondary State']] = median_income_df['State'].str.split("-", n = 1, expand=True) 


# Drop unnecessary columns
median_income_df.drop(columns=['index','extra','extra2','MSA','State'],inplace=True) 

# Drop duplicates -- first instance is the median and mean income by MSA
median_income_df.drop_duplicates(subset=['Primary MSA1','Primary State'], inplace=True) 

median_income_df['MSA']= median_income_df['Primary MSA1']+", "+median_income_df['Primary State']
median_income_df = median_income_df[["MSA", "Median income (dollars)", "Mean income (dollars)"]]

# Drop the name that would look as the name of the index otherwise
median_income_df.columns.name = None

#creating filtered dataframe for affordability calc
# median_income_affdf=median_income_df.filter(['Primary MSA','Median income (dollars)','Mean income (dollars)'], axis=1)
median_income_df=median_income_df.set_index("MSA")

# Further cleaning the columns median income and average income
median_income_df['Median income (dollars)'] =median_income_df['Median income (dollars)'].apply(lambda x: float(x.split()[0].replace(',', '')))
median_income_df['Mean income (dollars)']=median_income_df['Mean income (dollars)'].apply(lambda x: float(x.split()[0].replace(',', '')))

#  NOt in use any more
# median_income_df['Median income (dollars)']=median_income_df['Median income (dollars)'].astype(str).astype(float)
# median_income_df['Mean income (dollars)']=median_income_df['Mean income (dollars)'].astype(str).astype(float)
median_income_df.head()


,Median income (dollars),Mean income (dollars)
MSA,,
"Aberdeen, WA",61026.0,77948.0
"Abilene, TX",54808.0,71517.0
"Adrian, MI",53865.0,69443.0
"Aguadilla, PR",16311.0,25781.0
"Akron, OH",57158.0,80800.0


## Census Data Base
### Cleaning and Re arranging

In [135]:
# read the house_prices_msa.csv. this file will be used for sale price data
file_name='msa_pop_by_year.csv'
csv_file=Path(f"../Data/{file_name}")
census_df= pd.read_csv(csv_file)

# I drop rows that i do not need
census_df=census_df.loc[2:430]

# I rename field that will be used as column
census_df.iloc[0,0]='Identifier'

#grab the first row for the header
new_header = census_df.iloc[0] 

#take the data less the header row
census_df = census_df[1:] 

#set the header row as the df header and reset index
census_df.columns = new_header 
census_df.reset_index(inplace=True)

# Delete name of existing index
census_df.columns.name = None

# Drop unnecessary columns and rows
census_df.drop(columns=['index','Census','Estimates Base'],inplace=True) 

census_df.drop([1],inplace=True) 
# census_df.drop(['.In Metropolitan Statistical Area'],inplace=True) 

# Delete Rows containing Metro Divisions
census_df['Type']= census_df['Identifier'].str[-14:]
census_df.drop(census_df[census_df['Type']=='Metro Division'].index, inplace = True) 
census_df.drop(census_df[census_df['Identifier']=='.In Metropolitan Statistical Area'].index, inplace = True) 
census_df.drop(census_df[census_df['Identifier']=='.'].index, inplace = True) 

# Split MSA from text string - Used to Create City, State and 
census_df[['City','extra']] = census_df['Identifier'].str.split(", ", n = 1, expand=True) 
census_df['State'] = census_df['extra'].str[0:2]
census_df[['City1','extra1']] = census_df['City'].str.split(".", n = 1, expand=True) 
census_df[['City2','extra2']] = census_df['extra1'].str.split(".", n = 1, expand=True) 

# Rename columns and delete the ones we do not use
# Clean and rename
census_df.rename(columns={'City':'drop', 'City2':'City'}, inplace= True)
census_df.drop(columns=['drop','City1','extra','extra1','extra2'],inplace=True) 

# Final Split MSA from text string
census_df[['City','extra']] = census_df['City'].str.split("-", n = 1, expand=True) 
census_df.drop(columns=['extra'],inplace=True) 

# census_df['MSA'] = census_df['MSA'].str[1:]
census_df['MSA'] = census_df['City']+', '+census_df['State']

# census_df=census_df.set_index('')
census_df.sort_values(by='MSA')

# Fill NANs for Puerto Rico and United States
census_df.iloc[0,-1]='United States'
census_df.MSA[census_df.City=='United States'] = 'United States'
census_df.MSA[census_df.City=='Puerto Rico'] = 'Puerto Rico'

# Create data farme for census statistics (CAGR, Last, others by MSA)
census_stats_df=pd.DataFrame(census_df[['MSA','City','State']])
census_stats_df=census_stats_df.set_index('MSA')


# Final clean up of data frame census_df - Drop unused columns and re Arrange Columns
census_df.drop(columns=['Identifier','Type','City','State'], inplace=True)
census_df=census_df.set_index('MSA')
census_df=census_df.transpose()

# Delete name of existing index
census_df.columns.name = None



census_df.head(5)


,United States,"Abilene, TX","Akron, OH","Albany, GA","Albany, OR","Albany, NY","Albuquerque, NM","Alexandria, LA","Allentown, PA","Altoona, PA",...,"Yuma, AZ",Puerto Rico,"Aguadilla, PR","Arecibo, PR","Guayama, PR","Mayagüez, PR","Ponce, PR","San Germán, PR","San Juan, PR","Yauco, PR"
2010,"309,321,666","165,585","703,031","154,145","116,891","871,082","889,558","154,096","821,923","127,045",...,"197,127","3,721,525","338,958","199,212","84,057","115,929","261,924","137,630","2,347,932","106,937"
2011,"311,556,874","166,634","703,200","154,545","118,164","872,778","898,016","154,402","824,930","126,970",...,"202,841","3,678,732","335,118","197,379","82,811","113,915","257,719","136,415","2,322,216","104,954"
2012,"313,830,990","167,442","702,109","153,976","118,273","874,698","901,939","154,400","826,144","126,544",...,"202,475","3,634,488","330,752","195,493","82,192","111,845","253,345","135,185","2,295,419","102,970"
2013,"315,993,715","167,473","703,621","152,667","118,405","877,065","904,953","154,623","825,731","125,836",...,"202,420","3,593,077","326,684","194,023","81,430","109,938","249,504","133,955","2,269,908","101,054"
2014,"318,301,008","168,342","704,908","151,949","119,042","878,113","904,538","154,558","828,470","125,274",...,"204,054","3,534,874","321,451","191,124","80,239","107,458","244,760","132,195","2,233,776","98,695"


In [136]:
# Calculate CAGR: 2010-2020 
# Add a column with most recent date
# Create a column that i will drop with last_date - used to calculate number of periods
start_year=int(census_df.index[0])
end_year=int(census_df.index[-1])

# Replaceing "," for ''
    ## Option 1: 
initial_population=census_df.iloc[0,:].apply(lambda x: float(x.split()[0].replace(',', '')))
last_population=census_df.iloc[-1,:].apply(lambda x: float(x.split()[0].replace(',', '')))

    ## OPtion 2:
# initial_population=census_df.iloc[0,:].str.replace(',', '').astype(float)
# last_population=census_df.iloc[-1,:].str.replace(',', '').astype(float)

census_stats_df['Population 2019 (DO NOT USE)']=last_population
census_stats_df['Population 2010 (DO NOT USE)']=initial_population

# Calculations

census_stats_df['Population 10 year CAGR']=(last_population / initial_population)**(1/(end_year-start_year))-1
             
# Found the minimum data point after the GFC for each MSA
# Create a column that i will drop with last_date - used to calculate number of periods
# rent_stats_df['oldest_date']=rental_price_df.index[0] # i need to populate the right data point
# rent_stats_df['oldest_date1']=rental_price_df.apply(pd.Series.first_valid_index)
census_stats_df.head(5)

,City,State,Population 2019 (DO NOT USE),Population 2010 (DO NOT USE),Population 10 year CAGR
MSA,,,,,
United States,None,None,328239523.0,309321666.0,0.006618
"Abilene, TX",Abilene,TX,172060.0,165585.0,0.004271
"Akron, OH",Akron,OH,703479.0,703031.0,0.000071
"Albany, GA",Albany,GA,146726.0,154145.0,-0.005466
"Albany, OR",Albany,OR,129749.0,116891.0,0.011663


## Monte Carlo Simulations data
### Cleaning and Re arranging

In [137]:
# read the house_prices_msa.csv. this file will be used for sale price data
file_name='mc_output.csv'
csv_file=Path(f"../Data/{file_name}")
mc_output_df= pd.read_csv(csv_file)
mc_output_df['MSA']=mc_output_df['Unnamed: 0']+', '+mc_output_df['Unnamed: 1']
mc_output_df=mc_output_df.set_index('MSA')
mc_output_df.drop(columns=['Unnamed: 0', 'Unnamed: 1', 'lat', 'lng', 'population', 'firstzip'],inplace=True)
mc_output_df.head(5)

,1 Year Avg Return,3 Year Avg Return,5 Year Avg Return
MSA,,,
"Albany, GA",0.020838,0.021972,0.022676
"Forrest City, AR",0.012267,0.012868,0.013272
"Hannibal, MO",0.022298,0.023627,0.024375
"Shelton, WA",0.052263,0.055535,0.056912
"Hereford, TX",0.049562,0.052332,0.054101


## Crime Data Base
### Cleaning and Re arranging

In [138]:
# read the csv file. this file will be used for sale price data
file_name='crime-in-us-2015.csv'
csv_file=Path(f"../Data/{file_name}")
us_crime_raw= pd.read_csv(csv_file)

file_name='cities-over-250k.csv'
csv_file=Path(f"../Data/{file_name}")
us_crime_raw_250k= pd.read_csv(csv_file)

# Merging two data sets of crime
crime_df= pd.merge(us_crime_raw,us_crime_raw_250k,left_on='city',right_on='NAME',how='inner')

#  Create identifier, called MSA
crime_df['MSA']=crime_df['city']+', '+crime_df['STATE']

crime_df.rename(columns={'city':'City','STATE':'State','violent_crime':'Violent Crime','murder':'Murder','murder_rate':'Murder Rate'},inplace=True)

# Not in Use
cols_to_order=['City', 'State']
new_columns=cols_to_order + (crime_df.columns.drop(cols_to_order).tolist())
crime_df=crime_df[new_columns]
crime_df = crime_df.set_index('MSA')
crime_df.drop(columns=['pop_2015', 'NAME', 'latitude', 'longitude', 'POP_2010'],inplace=True)

# Reorder the state col to the second col
# crime_df = crime_df[['City', 'State', 'violent_crime', 'murder','murder_rate','MSA']]
# crime_df = crime_df.set_index('MSA')
crime_df.head(5)

,City,State,Violent Crime,Murder,Murder Rate
MSA,,,,,
"Chicago, IL",Chicago,IL,24663,478,17.517531
"New York, NY",New York,NY,50088,352,4.116545
"Baltimore, MD",Baltimore,MD,9542,344,55.372055
"Houston, TX",Houston,TX,21994,303,13.317388
"Detroit, MI",Detroit,MI,11846,295,43.818931


### Merging of 7 data sets: 
* Sales Data
* Rent Data
* Lat, Lon, Density Data
* Median / Mean Income
* Census Data
* Monte Carlo Simulations
* crime_df

In [139]:
# List of clean databases
# sale_stats_df
# rent_stats_df
# lon_lat_density_df

# Remove duplicate columns in merge
sale_clean=sale_stats_df.drop(columns=['City','State'])
rent_clean=rent_stats_df.drop(columns=['City','State'])
census_clean=census_stats_df.drop(columns=['City','State'])
crime_df=crime_df.drop(columns=['City','State'])

# Merge - Puter as we want to give enough flexibility to drop data later
combined_df= pd.merge(sale_clean,mc_output_df,how='outer',on='MSA')
combined_df = pd.merge(combined_df,rent_clean,how='outer',on='MSA')
combined_df = pd.merge(lon_lat_density_df,combined_df,how='outer',on='MSA')
combined_df = pd.merge(median_income_df,combined_df,how='outer',on='MSA')
combined_df = pd.merge(census_clean,combined_df,how='outer',on='MSA')
combined_df = pd.merge(crime_df,combined_df,how='outer',on='MSA')

# (combined_df[combined_df.index=='New York, NY']).sort_index()
# combined_df[combined_df['City_x']=='New York'].sort_index()
# combined_df[combined_df['State']=='NY'].sort_index()
combined_df


,Violent Crime,Murder,Murder Rate,Population 2019 (DO NOT USE),Population 2010 (DO NOT USE),Population 10 year CAGR,Median income (dollars),Mean income (dollars),City,State,...,5 Year Avg Return,% Rent Change Since Feb,% Rent Change Since Feb 2019,2019 Annualized Rent,Avg Monthly Rent 2019,Avg Monthly Rent Last 12M,Last 12M Annualized Rent,Monthly Rent Last,Rent CAGR since 2014-01,Rent Standard Deviation
MSA,,,,,,,,,,,,,,,,,,,,,
"Chicago, IL",24663.0,478.0,17.517531,9458539.0,9470634.0,-0.000142,75379.0,104357.0,Chicago,IL,...,0.017432,-0.003939,0.015464,21155.0,1762.916667,1775.083333,21301.0,1770.0,0.022398,0.006431
"New York, NY",50088.0,352.0,4.116545,19216182.0,18923407.0,0.001707,83160.0,122967.0,New York,NY,...,0.040052,-0.022710,0.015043,32189.0,2682.416667,2670.916667,32051.0,2625.0,0.020290,0.005360
"Baltimore, MD",9542.0,344.0,55.372055,2800053.0,2715738.0,0.003403,83160.0,110712.0,Baltimore,MD,...,0.031617,0.002401,0.010341,19856.0,1654.666667,1666.666667,20000.0,1670.0,0.014117,0.005929
"Houston, TX",21994.0,303.0,13.317388,7066141.0,5947236.0,0.019339,69193.0,99137.0,Houston,TX,...,0.029722,-0.001347,0.009537,17720.0,1476.666667,1484.583333,17815.0,1483.0,0.023289,0.003284
"Detroit, MI",11846.0,295.0,43.818931,4319629.0,4291508.0,0.000726,63474.0,86808.0,Detroit,MI,...,0.025083,0.016155,0.016598,14619.0,1218.250000,1242.750000,14913.0,1258.0,0.031806,0.008238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Logan, ID",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Weirton, OH",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Cañon City, CO",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.056886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
#Calculating affordability and adding it in combined DF
combined_df['Affordability Rent/Median Inc']=combined_df['2019 Annualized Rent']/combined_df['Median income (dollars)']
combined_df['Affordability Rent/Avg Inc']=combined_df['2019 Annualized Rent']/combined_df['Mean income (dollars)']
combined_df['Capitalization Rate Last']=(combined_df['Monthly Rent Last']*12)/combined_df['Sale Price Last']
combined_df.head(5)

,Violent Crime,Murder,Murder Rate,Population 2019 (DO NOT USE),Population 2010 (DO NOT USE),Population 10 year CAGR,Median income (dollars),Mean income (dollars),City,State,...,2019 Annualized Rent,Avg Monthly Rent 2019,Avg Monthly Rent Last 12M,Last 12M Annualized Rent,Monthly Rent Last,Rent CAGR since 2014-01,Rent Standard Deviation,Affordability Rent/Median Inc,Affordability Rent/Avg Inc,Capitalization Rate Last
MSA,,,,,,,,,,,,,,,,,,,,,
"Chicago, IL",24663.0,478.0,17.517531,9458539.0,9470634.0,-0.000142,75379.0,104357.0,Chicago,IL,...,21155.0,1762.916667,1775.083333,21301.0,1770.0,0.022398,0.006431,0.280648,0.202718,0.083783
"New York, NY",50088.0,352.0,4.116545,19216182.0,18923407.0,0.001707,83160.0,122967.0,New York,NY,...,32189.0,2682.416667,2670.916667,32051.0,2625.0,0.020290,0.005360,0.387073,0.261769,0.063369
"Baltimore, MD",9542.0,344.0,55.372055,2800053.0,2715738.0,0.003403,83160.0,110712.0,Baltimore,MD,...,19856.0,1654.666667,1666.666667,20000.0,1670.0,0.014117,0.005929,0.238769,0.179348,0.065134
"Houston, TX",21994.0,303.0,13.317388,7066141.0,5947236.0,0.019339,69193.0,99137.0,Houston,TX,...,17720.0,1476.666667,1484.583333,17815.0,1483.0,0.023289,0.003284,0.256095,0.178743,0.077856
"Detroit, MI",11846.0,295.0,43.818931,4319629.0,4291508.0,0.000726,63474.0,86808.0,Detroit,MI,...,14619.0,1218.250000,1242.750000,14913.0,1258.0,0.031806,0.008238,0.230315,0.168406,0.078108


## All the data bases: 
* ### Static:
 * rent_stats_df: Rent statistics by City and State (% Rent Change Since Feb, Avg Monthly Rent Last 12M, Last 12M Annualized Rent, Monthly Rent Last, Rent CAGR since 2014-01, Rent Standard Deviation)
 * lon_lat_density_df: lat/lon and populaiton information on each city / state City (State, Population, Density, Zip Code, lat,lon)
 * sale_stats_df:  Sale prices statistics by City and State (City, State, % Sale Price Change Since Feb, Avg Sale Price Last 12M, Sale Price Last, Sale Pr CAGR GFC, Sale Pr Standard Deviation)
 * median_income_df: Income info for each City / State (Median income (dollars)	Mean income (dollars))
 * census_stats_df: Population data for different MSAs (City, State, Population 2019, Population 2010, Population 10 year CAGR)
 * combined_df: Combines all of the above; performs an outer join in case we want to use different data for different purposes

* ### Historic:
 * rental_price_df: Historical information for rents since 2014
 * house_price_df: Historical information for sale prices since 2014
 * census_df: Annual population Estimates (2010-2020) for multiple MSAs



### Exporting clean data frames to csv for team to use in visualizations

In [141]:
#  Define a list of data frames, called df_list
df_list=[combined_df,rental_price_df,house_price_df]

#  Define a list of data frames names, called df_name
df_name=['combined_df','rental_price_df','house_price_df']

# All the data frames, in case needed, please le tthe team know.
# df_list=[rent_stats_df,lon_lat_density_df,sale_stats_df,median_income_df,census_stats_df,combined_df,rental_price_df,house_price_df,census_df]
# df_name=['rent_stats_df','lon_lat_density_df','sale_stats_df','median_income_df','census_stats_df','combined_df','rental_price_df','house_price_df','census_df']

# Create a for loop to iterate through each df and savethe files
for df, name in zip(df_list,df_name):
    file_name=name +'.csv'
    output_file = Path(f"../Data/Clean/{file_name}")
    df.to_csv(f"{output_file}")

In [142]:
# read the csv file. This file will be used for sale price data
# You don't need to do this each time as we are never changing the original data fram, called combined_df
file_name='combined_df.csv'
csv_file=Path(f"../Data/Clean/{file_name}")
combined_df= pd.read_csv(csv_file)

# Pulling a subset of data from combined df
# Min Population - leave equal to 0 if no conditions
min_population=0
columns_needed=['MSA','Avg Sale Price Last 12M', 'Sale Pr CAGR GFC','Population 10 year CAGR','Density','% Sale Price Change Since Feb','Rent CAGR since 2014-01','% Rent Change Since Feb','Capitalization Rate Last']
data_frame=combined_df[combined_df['Population']>min_population]
# data_frame.set_index('MSA', inplace=True)
data_frame=data_frame[columns_needed].dropna(subset=columns_needed)

data_frame.head(5)

,MSA,Avg Sale Price Last 12M,Sale Pr CAGR GFC,Population 10 year CAGR,Density,% Sale Price Change Since Feb,Rent CAGR since 2014-01,% Rent Change Since Feb,Capitalization Rate Last
0,"Chicago, IL",247694.583333,0.044707,-0.000142,4574.0,0.030465,0.022398,-0.003939,0.083783
1,"New York, NY",485123.166667,0.036593,0.001707,10715.0,0.031853,0.020290,-0.022710,0.063369
2,"Baltimore, MD",299185.000000,0.024738,0.003403,2830.0,0.036609,0.014117,0.002401,0.065134
3,"Houston, TX",223546.083333,0.054770,0.019339,1399.0,0.030610,0.023289,-0.001347,0.077856
4,"Detroit, MI",187385.833333,0.086075,0.000726,1864.0,0.040888,0.031806,0.016155,0.078108


### Using DataCompy to compare dataframes

In [143]:
#import statement for new python libary datacompy. This library compares 2 dataframes to see any differences or data gaps.

import datacompy

#resetting index of dataframe we want to compare
#Note to Team: when running this, please reset the index of the 2 dataframes you want to compare, and reset ONCE only.

sale_stats_df=sale_stats_df.reset_index()
rent_stats_df=rent_stats_df.reset_index()

#running the datacompy library

compare = datacompy.Compare(
sale_stats_df,
rent_stats_df,
join_columns='MSA',  #You can also specify a list of columns
abs_tol=0, #Optional, defaults to 0
rel_tol=0, #Optional, defaults to 0
df1_name='sale_stats_df', #Optional, defaults to 'df1' if no name specified
df2_name='rent_stats_df' #Optional, defaults to 'df2' if no name specified
)
print(compare.report())


DataComPy Comparison
--------------------

DataFrame Summary
-----------------

       DataFrame  Columns  Rows
0  sale_stats_df        9   914
1  rent_stats_df       12   106

Column Summary
--------------

Number of columns in common: 3
Number of columns in sale_stats_df but not in rent_stats_df: 6
Number of columns in rent_stats_df but not in sale_stats_df: 9

Row Summary
-----------

Matched on: msa
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 106
Number of rows in sale_stats_df but not in rent_stats_df: 808
Number of rows in rent_stats_df but not in sale_stats_df: 0

Number of rows with some compared columns unequal: 0
Number of rows with all compared columns equal: 106

Column Comparison
-----------------

Number of columns compared with some values unequal: 0
Number of columns compared with all values equal: 3
Total number of values which compare unequal: 0

Sample Rows Only in sale_stats_df (First 10 Columns)
---------